%pip install --upgrade astrapy

%pip install -q cassandra-driver
%pip install -q langchain
%pip install -q chat-deepseek-api
%pip install -q pypdf
%pip install -q cassio>=0.1.1

%pip install cassandra-driver

In [10]:
import cassandra

In [2]:
from cassandra.cluster import Cluster 

In [3]:
from cassandra.auth import PlainTextAuthProvider
import json

cloud_config= {'secure_connect_bundle':'secure-connect-pdf-qa (1).zip'}

with open("D:\PROJECTS\RagProject\yukkualahari@gmail.com-token (1).json") as f:
    secrets = json.load(f)
    
CLIENT_ID=secrets['clientId']
CLIENT_SECRET=secrets['secret']

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [4]:
row = session.execute("select release_version from system.local").one()

if row:
    print(row[0])
else:
    print("An error occurred.")

4.0.11-7cde36df13c4


%pip install langchain-deepseek-official langchain-community

In [12]:
from langchain_deepseek import ChatDeepSeek
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.docstore.document import Document
from langchain.chains import ConversationalRetrievalChain

%pip install langchain-openai

%pip install openrouter-python langchain-community

In [4]:
from langchain_community.llms import Ollama

In [5]:
llm = Ollama(
    base_url="http://localhost:11434",
    model="deepseek-r1:1.5b",
    temperature=0.3,
    num_gpu=1,  # Use GPU acceleration
    num_predict=128,
    num_ctx=800  # Use 1024 context tokens
)

C:\Users\yukku\AppData\Local\Temp\ipykernel_9700\392549622.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)


C:\Users\yukku\AppData\Local\Temp\ipykernel_9700\2996441843.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
d:\PROJECTS\RagProject\Rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
loader = PyPDFLoader("D:\PROJECTS\RagProject\how-to-win-friends-and-influence-people.pdf")
pages = loader.load_and_split()

In [11]:
len(pages)

192

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=30
)

In [7]:
keyspace = "pdf_qa_name"  
table_name = "pdf_qa_table"

In [13]:
vectorstore = Cassandra(
    embedding=embeddings,
    session=session,
    keyspace=keyspace,
    table_name=table_name
)

In [ ]:
texts = text_splitter.split_documents(pages)
vectorstore.add_documents(texts)

In [18]:
default_query = f"SELECT * FROM {keyspace}.{table_name}"
rows = session.execute(default_query)

In [ ]:
for idx, row in enumerate(rows):
    print(f"\nRow [{idx}]")
    print(f"row_id: {row.row_id}")
    print(f"embedding_vector: {str(row.vector)[:64]} ...")

In [19]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,  # Only this line changes from previous OpenAI setup
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

[Your PDFs] → [Cassandra Vector Store (unchanged)] 
                     ↓
[Local DeepSeek LLM] → [Conversational Chain (modified)]
                     ↑
           [Docker Container (ollama:11434)]

In [20]:
chat_history = []

In [ ]:
query = "Can you analyze the document and explain what is the big secret of dealing with people?"
result = qa_chain({"question": query, "chat_history": chat_history})
chat_history.append(result)

In [ ]:
print(result["answer"])

In order to get the most out of this book, devote a deep, driving desire to mastering the principles of human relations. By keeping a record of specific examples of how to apply these principles in practice, you will be inspired and motivated to make greater efforts in future endeavors. Additionally, by developing a strong desire to learn and improve your skills, you will be able to more effectively communicate with others and achieve better results in all areas of life.
